In [16]:
%pip install spacy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 17.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 42.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 11.7 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


In [22]:
from openai import OpenAI
from json import loads as json_loads
from json_repair import repair_json
from itertools import combinations

from nltk import download as nltk_download
nltk_download('punkt')
nltk_download('punkt_tab')
nltk_download('rslp')

from nltk import sent_tokenize as break_into_sentences
from nltk.stem import RSLPStemmer as stemmer

from re import search as re_search
import pandas as pd

import logging
import functools

from spacy.cli import download as spacy_download
spacy_download('pt_core_news_lg')

from spacy import load as spacy_load
nlp = spacy_load("pt_core_news_lg")

[nltk_data] Downloading package punkt to /home/tp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/tp/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to /home/tp/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 41.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
llm_client = OpenAI(
  api_key="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",
  base_url="https://integrate.api.nvidia.com/v1"
)

MODEL = "meta/llama-3.1-405b-instruct"

@functools.cache
def llm_inference(system_prompt:str, user_prompt:str):
    completion = llm_client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":user_prompt}
        ],
        temperature=0.7,
        top_p=0.7,
        max_tokens=2048,
        stream=False
    )
    
    return completion.choices[0].message.content

In [3]:
def extract_array(generated:str):
    return re_search(r"(\[[\W|\w|\s]*\])", generated).group(0)

In [4]:
knowledge_graph_prompt = """
Considando a frase abaixo, responda somente com o JSON abaixo, adicionando quantas entradas forem necessárias:
[
    {"entidade_origem": "Nome da entidade", "relacionamento": "Uma única palavra, verbo no infinitivo, que descreve o relacionamento entre as entidades", "entidade_destino": "Nome da entidade"},
]
"""

def add_to_relationships(item:dict, relationships:list):   
    origin_entity:list[str] = item["entidade_origem"]
    relationship:str = item["relacionamento"]
    destination_entity:list[str] = item["entidade_destino"]
    
    relationships.append((origin_entity, relationship, destination_entity))

@functools.cache
def infer_knowledge_graph(phrase:str) -> tuple[str,str,str]:
    generated = extract_array(
        llm_inference(knowledge_graph_prompt, phrase)
    )
    
    logging.info(generated)
    
    pair_relationships = []
    try:
        json_completion = json_loads(repair_json(generated))
        for item in json_completion:
            add_to_relationships(item, pair_relationships)
            
    except Exception as e:
        logging.error("FAILED TO GET GRAPH FROM COMPLETION")
        logging.error(e)
    
    return pair_relationships

In [5]:
entity_grouping_prompt = """
Considere a lista de entidades que será fornecida abaixo. Seu objetivo é agrupar as entidades sinônimas, ou seja, que se referem a uma mesma coisa, utilizando nomes diferentes, siglas, etc. Também agrupe ações sinônimas, ou seja, que descrevem a mesma ação, mas que são descritas de formas diferentes.
Responda EXCLUSIVAMENTE seguindo o formato JSON abaixo.
[
    {"principal": "Nome principal", "apelidos": ["Outros nomes ou palavras que devem ser agrupados com esta"]}
]
"""

@functools.cache
def infer_name_replacement(phrase:str) -> dict[str,str]:
    generated = extract_array(
        llm_inference(entity_grouping_prompt, phrase)
    )
    logging.warning(generated)
    
    to_replace = {}
    try:
        json_completion:list[dict[str,list[str]]] = json_loads(generated)
        for item in json_completion:
            main_entity = item["principal"]
            for other_name in item["apelidos"]:
                to_replace[other_name.lower().strip()] = main_entity.lower().strip()
    except Exception as e:
        logging.error("FAILED TO GET REPLACEMENTS FROM COMPLETION")
        logging.error(e)
    
    return to_replace

In [6]:
def text_to_graph(text:str, should_break:bool=False) -> pd.DataFrame:
    connections = []
    sentences = [text]
    
    if(should_break):
        sentences += break_into_sentences(text)
    
    for sentence in sentences:
        connections += infer_knowledge_graph(sentence)    

    return pd.DataFrame(connections, columns=["entity1", "relationship", "entity2"]).apply(lambda x: x.str.lower())

In [7]:
def unify_graph_terms(graphs: list[pd.DataFrame]) -> list[pd.DataFrame]:
    all_named_entities = []
    replacements = {}

    all_named_entities = pd.concat([graph.melt()["value"] for graph in graphs]).unique()

    replacements = infer_name_replacement(str(all_named_entities))
    graphs = [graph.map(lambda x: replacements.get(x.lower(), x).lower()).drop_duplicates() for graph in graphs]
    replaced_named = {replacements.get(name, name) for name in all_named_entities}

    return graphs, replaced_named, replacements

In [8]:
original = """De acordo com a pensadora brasileira Djamila Ribeiro, o primeiro passo a ser tomado para solucionar uma questão é tirá-la da invisibilidade. Porém, no contexto atual do Brasil, as mulheres enfrentam diversos desafios para que seu trabalho de cuidado seja reconhecido, gerando graves impactos em suas vidas, como a falta de destaque. Nesse sentido, essa problemática ocorre em virtude da omissão governamental e da influência midiática."""
original_graph = text_to_graph(original, True)
original_graph

,entity1,relationship,entity2
0,djamila ribeiro,analisar,questão
1,mulheres,enfrentar,desafios
2,desafios,gerar,impactos
3,governo,omitir,reconhecimento do trabalho de cuidado
4,mídia,influenciar,percepção do trabalho de cuidado
5,djamila ribeiro,defender,solução
6,questão,sofrer,invisibilidade
7,mulheres,enfrentar,desafios
8,desafios,gerar,impactos
9,impactos,afetar,vidas


In [9]:
paraphrase = """Segundo a filósofa brasileira Djamila Ribeiro, o ponto de partida para resolver um problema é torná-lo visível. No entanto, no cenário atual do Brasil, as mulheres enfrentam inúmeros obstáculos para que o trabalho de cuidado que realizam seja valorizado, o que causa sérias consequências em suas vidas, como a ausência de reconhecimento. Essa situação é resultado, em grande parte, da negligência do governo e da influência exercida pelos meios de comunicação."""

paraphrase_graph = text_to_graph(paraphrase, True)
paraphrase_graph

,entity1,relationship,entity2
0,djamila ribeiro,analisar,problema
1,mulheres,enfrentar,obstáculos
2,governo,negligenciar,trabalho de cuidado
3,meios de comunicação,influenciar,percepção do trabalho de cuidado
4,djamila ribeiro,afirmar,ponto de partida para resolver um problema
5,ponto de partida para resolver um problema,ser,tornar o problema visível
6,mulheres,enfrentar,obstáculos
7,obstáculos,causar,consequências
8,consequências,afetar,vidas das mulheres
9,governo,exercer,negligência


In [10]:
graphs, names, replacements = unify_graph_terms([original_graph, paraphrase_graph])

    {"principal": "trabalho de cuidado", "apelidos": ["cuidado", "reconhecimento do trabalho de cuidado", "percepção do trabalho de cuidado"]},
    {"principal": "governo", "apelidos": ["omissão governamental", "influência governamental"]},
    {"principal": "mídia", "apelidos": ["meios de comunicação", "influência midiática"]},
    {"principal": "problema", "apelidos": ["questão", "problemática", "obstáculos", "consequências"]},
    {"principal": "reconhecimento", "apelidos": ["reconhecimento do trabalho de cuidado", "ser reconhecido"]},
    {"principal": "influência", "apelidos": ["influenciar", "influência midiática", "influência governamental"]},
    {"principal": "negligenciar", "apelidos": ["omitir", "negligência"]},
    {"principal": "afetar", "apelidos": ["sofrer", "afetar", "causar"]},
    {"principal": "solução", "apelidos": ["ponto de partida para resolver um problema", "tornar o problema visível"]},
    {"principal": "desafios", "apelidos": ["impactos", "obstáculos", "conse

In [25]:
graphs[0].to_csv("original_graph.csv", index=False)

In [26]:
graphs[1].to_csv("paraphrase_graph.csv", index=False)

In [13]:
pd.merge(*graphs, how='inner')

,entity1,relationship,entity2
0,mulheres,analisar,problema
1,mulheres,enfrentar,desafios
2,mídia,influência,trabalho de cuidado


In [24]:
nlp(original).similarity(nlp(paraphrase))

0.9638450145721436